In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mlds

Mounted at /content/drive
/content/drive/MyDrive/mlds


In [2]:
import data_UCI as uci
import tasker_link_prediction as t_lp
from splitter import splitter
from models import TwoStream_GCN
from trainer import Trainer
import yaml
import os

In [3]:
prep = False

with open('config/config_UCI.yaml', 'r') as file:
    config = yaml.safe_load(file)

model_path = config['model_path']

if not os.path.exists(model_path):
    os.mkdir(model_path)
    os.mkdir(model_path + "best_checkpoints/")
    os.mkdir(model_path + "latest_checkpoints/")

with open(model_path + 'config.yaml', 'w') as file:
    yaml.dump(config, file)

config

{'adam_config': {'adam_beta_1': 0.9,
  'adam_beta_2': 0.999,
  'adam_epsilon': 1e-07,
  'adam_learning_rate': 0.005},
 'model_path': 'models/UCI_TwoStream/',
 'classifier_hidden_size': 24,
 'ffn_fusion_size': 12,
 'ffn_hidden_size': 12,
 'gcn_fusion_size': 24,
 'spatial_hidden_size': 24,
 'spatial_input_dim': 64,
 'temporal_hidden_size': 24,
 'temporal_input_dim': 60,
 'num_epochs': 1000,
 'patience': 10,
 'major_threshold': None,
 'train_proportion': 0.71,
 'dev_proportion': 0.1,
 'data_path': 'data/UCI/',
 'prep_data_path': 'prep_data/UCI_neg/'}

In [4]:
data = uci.UC_Irvine_Dataset(config['data_path'])
tasker = t_lp.Link_Pred_Tasker(data, path=config['prep_data_path'], prep=prep,
                               embs_dim=config['spatial_input_dim'], temp_dim=int(config['temporal_input_dim']/10),
                               major_threshold=config['major_threshold'], smart_neg_sampling=True)

splitter_as = splitter(tasker, train_proportion = config['train_proportion'], dev_proportion = config['dev_proportion'])

model= TwoStream_GCN(spatial_input_dim=config['spatial_input_dim'],
                 temporal_input_dim=config['temporal_input_dim'],
                 spatial_hidden_size=config['spatial_hidden_size'],
                 temporal_hidden_size=config['temporal_hidden_size'],
                 classifier_hidden_size=config['classifier_hidden_size'],
                 gcn_fusion_size=config['gcn_fusion_size'],
                 ffn_fusion_size=config['ffn_fusion_size'],
                 ffn_hiden_size=config['ffn_hidden_size'])

trainer = Trainer(model=model, splitter=splitter_as, model_path=model_path, adam_config=config['adam_config'], patience=config['patience'])
print(trainer.patience)

Dataset splits sizes:  train 53 dev 9 test 17
10


In [5]:
trainer.train(config['num_epochs'])

Epoch 1, Step 1, Loss: 0.2282620072364807
Epoch 1, Step 2, Loss: 0.12501214444637299
Epoch 1, Step 3, Loss: 0.08930474519729614
Epoch 1, Step 4, Loss: 0.07176508754491806
Epoch 1, Step 5, Loss: 0.06538151949644089
Epoch 1, Step 6, Loss: 0.05936218053102493
Epoch 1, Step 7, Loss: 0.05360637232661247
Epoch 1, Step 8, Loss: 0.04846390709280968
Epoch 1, Step 9, Loss: 0.05504963919520378
Epoch 1, Step 10, Loss: 0.03689223527908325
Epoch 1, Step 11, Loss: 0.025829272344708443
Epoch 1, Step 12, Loss: 0.025422798469662666
Epoch 1, Step 13, Loss: 0.0266325194388628
Epoch 1, Step 14, Loss: 0.021093660965561867
Epoch 1, Step 15, Loss: 0.02905338630080223
Epoch 1, Step 16, Loss: 0.04730558767914772
Epoch 1, Step 17, Loss: 0.02537008933722973
Epoch 1, Step 18, Loss: 0.0306964460760355
Epoch 1, Step 19, Loss: 0.010875536128878593
Epoch 1, Step 20, Loss: 0.01240849681198597
Epoch 1, Step 21, Loss: 0.012613222934305668
Epoch 1, Step 22, Loss: 0.012458082288503647
Epoch 1, Step 23, Loss: 0.011827181093

In [6]:
trainer.restore_best_checkpoint()
trainer.validate('Validation')
trainer.validate('Test')

Validation Metric MRRs: 0.28493468666771843
Validation Metric MAPs: 0.13544323071101488
Test Metric MRRs: 0.3749966502061407
Test Metric MAPs: 0.2158253059900545


(0.3749966502061407, 0.2158253059900545)